<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_api_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install required libraries

In [ ]:
!pip install flask # webserver 
!pip install faker # fake data generator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# create the API app in flask

In [ ]:
from flask import Flask, request, jsonify
import sqlite3
import threading
import multiprocessing, time

app = Flask(__name__)

DATABASE = 'customers.db'

@app.route("/")
def home():
    return "Hello World! This is the API homepage. No swagger."


def create_table():
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT NOT NULL,
            phone TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

@app.route('/customers', methods=['GET'])
def get_customers():
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM customers")
    customers = cursor.fetchall()
    conn.close()
    return jsonify(customers)

@app.route('/customers/<int:customer_id>', methods=['GET'])
def get_customers(customer_id):
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM customers where customerid = ?",customer_id)
    customers = cursor.fetchall()
    conn.close()
    return jsonify(customers)

@app.route('/customers', methods=['POST'])
def add_customer():
    name = request.json.get('name')
    email = request.json.get('email')
    phone = request.json.get('phone')

    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO customers (name, email, phone) VALUES (?, ?, ?)", (name, email, phone))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Customer added successfully'})

@app.route('/customers/<int:customer_id>', methods=['PUT'])
def update_customer(customer_id):
    name = request.json.get('name')
    email = request.json.get('email')
    phone = request.json.get('phone')

    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("UPDATE customers SET name=?, email=?, phone=? WHERE id=?", (name, email, phone, customer_id))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Customer updated successfully'})

@app.route('/customers/<int:customer_id>', methods=['DELETE'])
def delete_customer(customer_id):
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM customers WHERE id=?", (customer_id,))
    conn.commit()
    conn.close()

    return jsonify({'message': 'Customer deleted successfully'})

if __name__ == '__main__':
    create_table()
    multiprocessing.Process(target=app.run).start() # this is required for background running so the cell can be released.
    


AssertionError: ignored

# get public url

use the url here to make sure flask is running. you should get a webpage that says '*Hello World! This is the API homepage.*'

In [ ]:
# this cell outputs the url you can use in your browser to see if the website/api is running. 

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://rqf73bmigk-496ff2e9c6d22116-5000-colab.googleusercontent.com/


## function: get customers

In [ ]:
def get_customers():

  import requests

  url = f'http://localhost:5000/customers'

  response = requests.get(url)
  customers = response.json()

  if response.status_code == 200:
      return customers  
  else:
      return(f'Error: {response.status_code} - {response.text}')

  return customers  


def get_customer(customerid):

  import requests

  url = f'http://localhost:5000/customers/{customerid}'

  response = requests.get(url)
  customer = response.json()

  if response.status_code == 200:
      return customer  
  else:
      return(f'Error: {response.status_code} - {response.text}')

  return customer  


# function: generate fake customer 

generate fake customer, post to the API which inserts the record into sqlite.

In [ ]:
def generate_customers():

  import requests
  from faker import Faker

  url = 'http://localhost:5000/customers'

  faker = Faker()
  
  new_customer = {
      'name': faker.name(),
      'email': faker.email(),
      'phone': faker.phone_number()
  }

  response = requests.post(url, json=new_customer)

  if response.status_code == 200:
      print('New customer added successfully.')
  else:
      print(f'Error: {response.status_code} - {response.text}')


# generate a new customer record and post it to the API

In [ ]:
generate_customers()

New customer added successfully.


# get all customer records with the API

In [ ]:
get_customers()

[[1, 'Patrick Miles', 'frankjohnson@example.org', '+1-971-760-8216x184'],
 [2, 'Teresa Thomas', 'dmartinez@example.net', '001-550-861-1522x028'],
 [3, 'Holly Brown', 'lisawalsh@example.org', '001-984-959-9303'],
 [4, 'Jordan Pham', 'dsantana@example.org', '(354)856-8655x76442']]

In [ ]:
get_customer(1)

JSONDecodeError: ignored

# query the database with pandas

In [ ]:
import pandas as pd

In [ ]:
DATABASE = 'customers.db'

In [ ]:
conn = sqlite3.connect(DATABASE)

In [ ]:
pd.read_sql_query("select * from customers",conn)

,id,name,email,phone
0,1,Patrick Miles,frankjohnson@example.org,+1-971-760-8216x184
1,2,Teresa Thomas,dmartinez@example.net,001-550-861-1522x028
2,3,Holly Brown,lisawalsh@example.org,001-984-959-9303
3,4,Jordan Pham,dsantana@example.org,(354)856-8655x76442


## kill the webserver


the second pid which has the first pid as it's parent process is the one to kill

In [ ]:
!ps -ef | grep ipykernel_launcher

root        8946      83  0 17:19 ?        00:00:05 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-bfa8356a-25e0-4aa6-8c2b-6df44cb4c8bd.json
root        9870    8946  0 17:22 ?        00:00:00 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-bfa8356a-25e0-4aa6-8c2b-6df44cb4c8bd.json
root       11257    8946  0 17:28 ?        00:00:00 /bin/bash -c ps -ef | grep ipykernel_launcher
root       11259   11257  0 17:28 ?        00:00:00 grep ipykernel_launcher


In [ ]:
!kill -9 9870